In [514]:
import requests as r
import brotli
import bs4
import re
import json
import random
rgx = re.compile('(\d+)\sшт\sпо\s(\d+)')

import pandas as pd

In [525]:
order_details_selector = 'section.tf9_14 > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)'
order_detail_products_selector = ".e4"
product_selector = ".n5f_14"
product_name_selector = ".of1_14"
product_link_selector = ".of1_14"
price_selector =        "div:nth-child(3) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)" 
count_selector = ".of2_14"
price_details_selector = "div.fn6_14:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > ul:nth-child(2) > li:nth-child(1) > div:nth-child(3) > div:nth-child(2)"
order_name_selector = ".n3f_14 h2"
order_date_selector = ".fn4_14 p"
state_selector = ".e6r_14"
ship_selector = ".fo8_14"
order_price_selector = ".f0w_14"

In [532]:
def parse_products(soup):
    #text = brotli.decompress(res.content).decode('utf-8')
    #soup = bs4.BeautifulSoup(text)
    results = {}
    ships = soup.select(ship_selector)
    for ship in ships:
        state = ship.select_one(state_selector).text.strip()
        products = ship.select(product_selector)
        order_name = soup.select_one(order_name_selector).text
        if(len(products) == 0):
            print(f"order {order_name} have no products.")
        for product in products:
            text = product.select_one(product_name_selector).text
            link = product.select_one(product_link_selector).attrs['href']
            total_price = product.select_one(price_selector).text.replace('\xa0', '')
            count = 1
            price = int(total_price.split()[0])
            
            count_el = product.select_one(count_selector)
            if count_el:
                if m := rgx.match(count_el.text.replace('\xa0', '')):
                    count = int(m.groups()[0])
                    price = int(m.groups()[1])
            if results.get(link):
                c = results[link]["count"]
                results[link]["count"] = c + count
                results[link]["total_price"] = (count+c) * price
            else:
                results[link] = dict(total_price=price, text=text, count=count, price=price, state=state)
        return results

- get order links from browser
```JS
orders = $$('section.tf9_14 > div > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) ').map(e => [e.children[0].innerText, e.children[1].children[0].href])
JSON.stringify(orders)
```
- copy cookies (raw) from dev tab when you logged-in
- this script produce dataFrame/excel with price, date, description
- fill each line with fund name
- run other part to split rows by fund in separated lists
- upload to the suckmyfunds excel manually

In [415]:
orders = json.loads("""
[["Заказ от 26 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0331"],["Заказ от 14 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0315"],["Заказ от 21 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0323"],["Заказ от 20 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0321"],["Заказ от 9 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0311"],["Заказ от 24 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0330"],["Заказ от 23 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0329"],["Заказ от 23 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0327"],["Заказ от 23 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0328"],["Заказ от 21 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0326"],["Заказ от 21 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0324"],["Заказ от 21 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0325"],["Заказ от 20 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0322"],["Заказ от 16 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0318"],["Заказ от 17 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0320"],["Заказ от 15 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0316"],["Заказ от 13 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0313"],["Заказ от 17 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0319"],["Заказ от 16 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0317"],["Заказ от 14 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0314"],["Заказ от 22 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0297"],["Заказ от 10 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0312"],["Заказ от 4 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0309"],["Заказ от 4 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0310"],["Заказ от 3 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0308"],["Заказ от 3 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0307"],["Заказ от 2 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0306"],["Заказ от 30 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0304"],["Заказ от 1 июля","https://www.ozon.ru/my/orderdetails/?order=47357488-0305"],["Заказ от 29 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0303"],["Заказ от 27 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0302"],["Заказ от 26 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0301"],["Заказ от 24 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0299"],["Заказ от 22 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0296"],["Заказ от 24 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0300"],["Заказ от 23 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0298"],["Заказ от 19 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0292"],["Заказ от 20 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0295"],["Заказ от 19 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0293"],["Заказ от 19 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0294"],["Заказ от 18 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0291"],["Заказ от 15 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0288"],["Заказ от 17 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0290"],["Заказ от 16 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0289"],["Заказ от 12 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0286"],["Заказ от 11 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0262"],["Заказ от 12 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0287"],["Заказ от 5 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0281"],["Заказ от 9 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0284"],["Заказ от 10 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0285"],["Заказ от 5 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0282"],["Заказ от 31 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0277"],["Заказ от 7 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0283"],["Заказ от 4 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0280"],["Заказ от 2 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0279"],["Заказ от 1 июня","https://www.ozon.ru/my/orderdetails/?order=47357488-0278"],["Заказ от 28 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0276"],["Заказ от 22 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0270"],["Заказ от 26 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0274"],["Заказ от 26 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0273"],["Заказ от 28 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0275"],["Заказ от 23 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0271"],["Заказ от 21 мая","https://www.ozon.ru/my/orderdetails/?order=47357488-0268"]]
""")

In [473]:
s = r.session()

raw_headers = """Host: www.ozon.ru
User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:128.0) Gecko/20100101 Firefox/128.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate, br, zstd
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: none
Sec-Fetch-User: ?1
Connection: keep-alive
Cookie: abt_data=b1d38aa349c54b55098401b831b5253d:448f6a89a4155ed9bae2bff330f6ea91eb97b5931210dead29651dee623329c1756ce4105649f92db7da5b1de738347ed27c25d2e55612354386e3afc03d9b97e854f14c30f09fd97b7ec876275cf05b19414a48a01c51da9908e06ea520346cec7cb8095e566e44824e5ae01bb7f70f1b7bf72633814594ae091fe1cce4fd9bc7432bda82640b5153a743e31ddd05badcf662472a3223193625fcdd2e47ee0cf1f7968a46df63cfa29c34b2b2c20b1c931d9ea9b50158e081f2dda66867b1f62a3ccd5578e61fb618909d37079cc18b5f1399bc1297f83fb5ef5cf295ee33932f94a9ac68ed19269c6949b84a995f8a0baa733785a97f31444f5faa653acc82237c3c6d7a4ebb29d440e3c9e04324eec17d98f487f421ae4249e25c27224371ac57b5f6bbc6e78745027470cf2bdecbd83caddf3dcf95d86072a20f92ec172cd4a0646646a5f526516e9f06e9b38de436e633e7d2870f05f9c46969009183008d751b207debe79975ac85a0112148ee; __Secure-ext_xcid=a10905cb2ab5e0f623fbae11f36fa8c0; __Secure-user-id=47357488; __Secure-access-token=5.47357488.qqoNqh_oQgeanBEBl5VHtA.25.AeXHuyWxwzKs7-0xRvzX-2WW4VFAbGFEZP-gljp40yBIdOabVd-Usm2G4ia4tcGmns34y6HdwcgfV9N5EcI3-lA.20200608152127.20240804195717.NsoFWHI-vwiV6qykGu-YUuWYEeXwlPcN71J7UN8SBOE.16cb94e8d025d47ff; __Secure-ab-group=25; __Secure-refresh-token=5.47357488.qqoNqh_oQgeanBEBl5VHtA.25.AeXHuyWxwzKs7-0xRvzX-2WW4VFAbGFEZP-gljp40yBIdOabVd-Usm2G4ia4tcGmns34y6HdwcgfV9N5EcI3-lA.20200608152127.20240804195717.YQmS4TApVLRVfJXoNysjj2RihsllQ5LGiH_wuc9H3wQ.1f380b45a50df55ac; __Secure-ETC=2cb26b6bd7dd01f61d5da2c7ddaf9367; cf_clearance=9mRI8nTq39f42DxfD9vNwyxZwCoQzPRtf1ENIhiVl08-1715678422-1.0.1.1-B1sxd9ReDDnVA4z0dK6hyzciogYcFJQj1FGoKhmOh95lGtIkvcweYOTTKjcXIZlxATqujRaUjQqajp0lEwv9IA; xcid=3ea9c26db2cb8bb00b2a3f4e341b072a; nlbi_1101384=6c2OSelbJGRAgYysK8plmQAAAACCOteUko7VY2X2/YPpIS5l; feedback_id_22=1; AREA_ID=2; incap_ses_586_1101384=spFSXTTkiGQew0EhsuQhCLNWG2IAAAAAWVEeJqkXBPagM1SBOD9rKg==; is_adult_confirmed=; is_alco_adult_confirmed=; guest=true; ADDRESSBOOKBAR_WEB_CLARIFICATION=1722589628; rfuid=LTE5NTAyNjU0NzAsMzUuNzQ5OTcyMDkzODUwMzc0LC03NzI5NzIxMTAsTGludXggeDg2XzY0LC03MDk1MDEwMjgsVzNzaWJtRnRaU0k2SWxCRVJpQldhV1YzWlhJaUxDSmtaWE5qY21sd2RHbHZiaUk2SWxCdmNuUmhZbXhsSUVSdlkzVnRaVzUwSUVadmNtMWhkQ0lzSW0xcGJXVlVlWEJsY3lJNlczc2lkSGx3WlNJNkltRndjR3hwWTJGMGFXOXVMM0JrWmlJc0luTjFabVpwZUdWeklqb2ljR1JtSW4wc2V5SjBlWEJsSWpvaWRHVjRkQzl3WkdZaUxDSnpkV1ptYVhobGN5STZJbkJrWmlKOVhYMHNleUp1WVcxbElqb2lRMmh5YjIxbElGQkVSaUJXYVdWM1pYSWlMQ0prWlhOamNtbHdkR2x2YmlJNklsQnZjblJoWW14bElFUnZZM1Z0Wlc1MElFWnZjbTFoZENJc0ltMXBiV1ZVZVhCbGN5STZXM3NpZEhsd1pTSTZJbUZ3Y0d4cFkyRjBhVzl1TDNCa1ppSXNJbk4xWm1acGVHVnpJam9pY0dSbUluMHNleUowZVhCbElqb2lkR1Y0ZEM5d1pHWWlMQ0p6ZFdabWFYaGxjeUk2SW5Ca1ppSjlYWDBzZXlKdVlXMWxJam9pUTJoeWIyMXBkVzBnVUVSR0lGWnBaWGRsY2lJc0ltUmxjMk55YVhCMGFXOXVJam9pVUc5eWRHRmliR1VnUkc5amRXMWxiblFnUm05eWJXRjBJaXdpYldsdFpWUjVjR1Z6SWpwYmV5SjBlWEJsSWpvaVlYQndiR2xqWVhScGIyNHZjR1JtSWl3aWMzVm1abWw0WlhNaU9pSndaR1lpZlN4N0luUjVjR1VpT2lKMFpYaDBMM0JrWmlJc0luTjFabVpwZUdWeklqb2ljR1JtSW4xZGZTeDdJbTVoYldVaU9pSk5hV055YjNOdlpuUWdSV1JuWlNCUVJFWWdWbWxsZDJWeUlpd2laR1Z6WTNKcGNIUnBiMjRpT2lKUWIzSjBZV0pzWlNCRWIyTjFiV1Z1ZENCR2IzSnRZWFFpTENKdGFXMWxWSGx3WlhNaU9sdDdJblI1Y0dVaU9pSmhjSEJzYVdOaGRHbHZiaTl3WkdZaUxDSnpkV1ptYVhobGN5STZJbkJrWmlKOUxIc2lkSGx3WlNJNkluUmxlSFF2Y0dSbUlpd2ljM1ZtWm1sNFpYTWlPaUp3WkdZaWZWMTlMSHNpYm1GdFpTSTZJbGRsWWt0cGRDQmlkV2xzZEMxcGJpQlFSRVlpTENKa1pYTmpjbWx3ZEdsdmJpSTZJbEJ2Y25SaFlteGxJRVJ2WTNWdFpXNTBJRVp2Y20xaGRDSXNJbTFwYldWVWVYQmxjeUk2VzNzaWRIbHdaU0k2SW1Gd2NHeHBZMkYwYVc5dUwzQmtaaUlzSW5OMVptWnBlR1Z6SWpvaWNHUm1JbjBzZXlKMGVYQmxJam9pZEdWNGRDOXdaR1lpTENKemRXWm1hWGhsY3lJNkluQmtaaUo5WFgxZCxXeUpsYmkxVlV5SXNJbVZ1TFZWVElpd2laVzRpWFE9PSwwLDEsMCwyNCwyMzc0MTU5MzAsLTEsMjI3MTI2NTIwLDAsMSwwLC00OTEyNzU1MjMsSUU1bGRITmpZWEJsSUVkbFkydHZJRXhwYm5WNElIZzRObDgyTkNBMUxqQWdLRmd4TVNrZ01qQXhNREF4TURFZ1RXOTZhV3hzWVE9PSxlMzA9LDY1LC0xMjg1NTUxMywxLDEsLTEsMTY5OTk1NDg4NywxNjk5OTU0ODg3LDMzNjAwNzkzMywxNg==
"""
headers = {l.split(':')[0].strip(): ":".join(l.split(":")[1:]).strip() for l in raw_headers.splitlines()}

for k,v in headers.items():
    s.headers[k] = v

#headers

In [475]:
result = dict()
datas = []

for order_info, order_link in orders:     
    print("go over", order_info, order_link)
    res = s.get(order_link)
    if res.status_code != 200:
        print(res.text)
        print(res.headers)
    else:
        text = brotli.decompress(res.content).decode('utf-8')
        soup = bs4.BeautifulSoup(text)
        datas.append(soup)
        #result[order_link] = parse_products(res)
    delay = 0.1
    time.sleep(delay)
        
#print(result)

go over Заказ от 26 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0331
go over Заказ от 14 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0315
go over Заказ от 21 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0323
go over Заказ от 20 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0321
go over Заказ от 9 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0311
go over Заказ от 24 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0330
go over Заказ от 23 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0329
go over Заказ от 23 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0327
go over Заказ от 23 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0328
go over Заказ от 21 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0326
go over Заказ от 21 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0324
go over Заказ от 21 июля https://www.ozon.ru/my/orderdetails/?order=47357488-0325
go over Заказ от 

In [486]:
datas[0]

'47357488-0331'

In [539]:
result = dict()
for soup in datas:
    order = soup.select_one(order_name_selector).text.strip().split()[-1]
    order_date = soup.select_one(order_date_selector).text.strip()
    order_price = soup.select_one(order_price_selector).text.strip() 
    order_price = int("".join(order_price.split()[:-1]))
    result[order] = dict(products=parse_products(soup), date=order_date, order_price=order_price)

In [544]:
frame = []
for order_name, order_data in result.items():
    for product_link, product_data in order_data["products"].items():
        
        frame.append(
            dict(
                name=order_name, 
                date=order_data["date"], 
                product_link=product_link, 
                total_price=product_data["total_price"],    
                order_pcire=order_data["order_price"],
                product_name=product_data["text"],
                count=product_data["count"],
                product_price=product_data["price"],
                state=product_data["state"],
            ))
                
df = pd.DataFrame.from_dict(frame)


,name,date,product_link,total_price,order_pcire,product_name,count,product_price,state
0,47357488-0331,от 26 июля,/product/925514455/,1070,1070,"Лента транспортерная для ворот h-200 мм, 4 м /...",1,1070,Отменён
1,47357488-0315,от 14 июля,/product/897279764/,351,1215,Скоба строительная Прямоугольная 2 шт.,3,351,Получен
2,47357488-0323,от 21 июля,/product/1538426771/,1256,12560,Сухой корм BOWL WOW для собак средних пород с ...,2,1256,Получен
3,47357488-0321,от 20 июля,/product/1313324061/,756,6979,"Сухофрукты Финики без сахара Мазафати, Полезно...",1,756,Получен
4,47357488-0311,от 9 июля,/product/1051248091/,3726,19290,Камера видеонаблюдения уличная 4G на солнечной...,5,3726,Отменён
...,...,...,...,...,...,...,...,...,...
58,47357488-0274,от 26 мая,/product/1196511729/,1751,2154,"Комплект капельного полива,Пластик",1,1751,Получен
59,47357488-0273,от 26 мая,/product/628198568/,1342,4817,Mr.Logo Секатор,1,1342,Получен
60,47357488-0275,от 28 мая,/product/881702737/,957,4795,"Точилка для ножей , RUIXIN PRO RX-008",2,957,Получен
61,47357488-0271,от 23 мая,/product/533052663/,184,7719,Муфта 3/4 соединительная для шланга Polyagro,2,184,Получен


In [524]:
df.to_excel("./parsed.xlsx")

In [551]:
soups = {}
for d in datas:
    soups[d.select_one(order_name_selector).text.strip().split()[-1]] = d

In [558]:
[p.text for p in soups["47357488-0327"].select(product_name_selector)]

['Кинезио тейп Kinexib CLASSIC, тейпы для тела, тейп для лица 5м*5см, синий',
 'CURAPROX Подарочный набор для ухода за гигиеной полости рта для взрослых и детей, в составе зубные щетки, пасты, нить и таблетки для индикации налета',
 'Пищевые волокна Фибраксин пребиотические, порошок в саше 15 штук по 6 г']